
# Assignment II: CUDA Basics I
## [DD2360] Applied GPU Programming
Group 8

Martin Forslund (uz6@kth.se)
| Valeria Grotto (vgrotto@kth.se)

***

# Exercise 1 - Your first CUDA program and GPU performance metrics
This exercise will get you familiar with the basic structure of a simple CUDA code. You will learn how to compile and run CUDA-based programs. You will practice memory allocation in host memory and device memory, the data movement between CPU and GPU, and the distribution of the computational threads in CUDA. You will also use Nvidia Nsight (see the tutorial on [Nsight profiling](https://canvas.kth.se/courses/42842/pages/tutorial-nvidia-nsight-systems-for-profiling?module_item_id=735390)) to profile your program.

When comparing the output between CPU and GPU implementation, the precision of the floating-point operations might differ between different versions, which can translate into rounding error differences. Hence, use a margin error range when comparing both versions.

Please implement a simple vectorAdd program that sums two vectors and stores the results into a third vector. You will understand how to index 1D arrays inside a GPU kernel. Please complete the following main steps in your code. You can create your own code, or, use the following [code template](https://canvas.kth.se/courses/42842/files/6753924?wrap=1) and edit code parts demarcated by the //@@ comment lines.


* Allocate host and device memory
* Initialize host memory and create a reference result in CPU memory
* Copy from host memory to device memory
* Initialize thread block and kernel grid dimensions
* Invoke CUDA kernel
* Copy results from GPU to CPU
* Compare the results with the reference result
* Free host and device memory
* Write the CUDA kernel

***
### Questions to answer in the report

* Explain how the program is compiled and run.
* For a vector length of N:
  * How many floating operations are being performed in your vector add kernel?
  * How many global memory reads are being performed by your kernel?
* For a vector length of 1024:
  * Explain how many CUDA threads and thread blocks you used.
* Profile your program with Nvidia Nsight.
  * What Achieved Occupancy did you get? You might find https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html#nvprof-metric-comparisonLinks to an external site. useful.
* Now increase the vector length to 131070:
  * Did your program still work? If not, what changes did you make?
  * Explain how many CUDA threads and thread blocks you used.
* Profile your program with Nvidia Nsight. What Achieved Occupancy do you get now?
  * Further increase the vector length (try 6-10 different vector length), plot a stacked bar chart showing the breakdown of time including (1) data copy from host to device (2) the CUDA kernel (3) data copy from device to host. For this, you will need to add simple CPU timers to your code regions.




In [ ]:
%%writefile vectorAdd.cu
#include <stdio.h>
#include <sys/time.h>

// libraries to generate random numbers in C
#include <time.h>
#include <stdlib.h>

#include <cstdlib> //to convert char* to int

// libraries to generate random numbers in CUDA
#include <curand_kernel.h>
#include <curand.h>

#define DataType double
#define max_number 100
#define minimum_number 0

__global__ void vecAdd(DataType *in1, DataType *in2, DataType *out, int len) {
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    if(id < len) out[id] = in1[id] + in2[id];
}

//@@ Insert code to implement timer start
//@@ Insert code to implement timer stop
DataType cpuSecond() {
   struct timeval tp;
   gettimeofday(&tp,NULL);
   return ((DataType)tp.tv_sec + (DataType)tp.tv_usec*1.e-6);
}


int main(int argc, char **argv) {

  srand(time(NULL));   // Initialization for random numbers, should only be called once.

  int inputLength;
  DataType *hostInput1;
  DataType *hostInput2;
  DataType *hostOutput;
  //DataType *resultRef;
  DataType *deviceInput1;
  DataType *deviceInput2;
  DataType *deviceOutput;

  //@@ Insert code below to read in inputLength from args
  // decode arguments
  if(argc < 2) {
      printf("You must provide at least one argument\n");
      exit(0);
  } else {
    inputLength = atoi(argv[1]);
  }

  printf("The input length is %d\n", inputLength);

  size_t size = inputLength * sizeof(DataType);

  //@@ Insert code below to allocate Host memory for input and output
  hostInput1 = (DataType *) malloc(size);
  hostInput2 = (DataType *) malloc(size);
  hostOutput = (DataType *) malloc(size);

  //@@ Insert code below to initialize hostInput1 and hostInput2 to random numbers, and create reference result in CPU
  for (int i = 0; i < inputLength; i++) {
    // generate a pseudo-random integer between minimum_number and max_number
    hostInput1[i] = rand() % (max_number + 1 - minimum_number) + minimum_number;
    hostInput2[i] = rand() % (max_number + 1 - minimum_number) + minimum_number;
  }

  //@@ Insert code below to allocate GPU memory here
  cudaMalloc((void **)&deviceInput1, size);
  cudaMalloc((void **)&deviceInput2, size);
  cudaMalloc((void **)&deviceOutput, size);

  //@@ Insert code to below to Copy memory to the GPU here
  //cudaMemcpy(void* dst, const void* src, size_t count, cudaMemcpyKind kind )
  cudaMemcpy(deviceInput1, hostInput1, size, cudaMemcpyHostToDevice);
  cudaMemcpy(deviceInput2, hostInput2, size, cudaMemcpyHostToDevice);

  //@@ Initialize the 1D grid and block dimensions here
  int dimx = 32;
  dim3 block(dimx, 1);
  dim3 grid((inputLength + block.x - 1) / block.x, 1);

  //@@ Launch the GPU Kernel here
  DataType start = cpuSecond();

  vecAdd<<<grid, block>>>(deviceInput1, deviceInput2, deviceOutput, inputLength);
  cudaDeviceSynchronize();

  DataType elapsed = cpuSecond() - start;

  printf("vecAdd<<<(%d,%d), (%d,%d)>>> elapsed %f sec\n", grid.x, grid.y, block.x, block.y, elapsed);

  //@@ Copy the GPU memory back to the CPU here
  cudaMemcpy(hostOutput, deviceOutput, size, cudaMemcpyDeviceToHost);

  //@@ Insert code below to compare the output with the reference
  double tolerance = 1.0e-14;
  DataType expeted;
  for(int i=0; i<inputLength; i++)
  {
    expeted = hostInput1[i] + hostInput2[i];
    //if the absolute value is greater than the tolerance we have an error
    if(fabs(hostOutput[i] - expeted) > tolerance)
    {
        printf("\nError: value of hostOutput[%d] = %f instead of %f\n\n", i, hostOutput[i], expeted);
        exit(1);
    } else {
      //printf("\nOk: value of hostOutput[%d] = %f - expected: %f\n\n", i, hostOutput[i], expeted);
    }
  }

  //@@ Free the GPU memory here
  cudaFree(deviceInput1);
  cudaFree(deviceInput2);
  cudaFree(deviceOutput);


  //@@ Free the CPU memory here
  free(hostInput1);
  free(hostInput2);
  free(hostOutput);

  printf("\n---------------------------------------------\n");
  printf("SUCCESS\n");

  return 0;
}


Writing vectorAdd.cu


The same version of vectorAdd but with a timer added.



In [ ]:
%%writefile vectorAdd_2.cu

#include <stdio.h>
#include <sys/time.h>
#include <time.h>
#include <stdlib.h>
#include <cstdlib> //to convert char* to int
#include <curand_kernel.h>
#include <curand.h>

#define DataType double
#define max_number 100
#define minimum_number 0

__global__ void vecAdd(DataType *in1, DataType *in2, DataType *out, int len) {
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    if(id < len) out[id] = in1[id] + in2[id];
}

DataType cpuSecond() {
   struct timeval tp;
   gettimeofday(&tp, NULL);
   return ((DataType)tp.tv_sec + (DataType)tp.tv_usec * 1.e-6);
}

//@@ Insert code to implement timer start
DataType timerStart;
DataType timerStop;

int main(int argc, char **argv) {
  srand(time(NULL));   // Initialization for random numbers, should only be called once.

  int inputLength;
  DataType *hostInput1;
  DataType *hostInput2;
  DataType *hostOutput;
  DataType *deviceInput1;
  DataType *deviceInput2;
  DataType *deviceOutput;

  //@@ Insert code below to read in inputLength from args
  // decode arguments
  if (argc < 2) {
      printf("You must provide at least one argument\n");
      exit(0);
  } else {
    inputLength = atoi(argv[1]);
  }

  printf("The input length is %d\n", inputLength);

  size_t size = inputLength * sizeof(DataType);

  //@@ Insert code below to allocate Host memory for input and output
  hostInput1 = (DataType *)malloc(size);
  hostInput2 = (DataType *)malloc(size);
  hostOutput = (DataType *)malloc(size);

  //@@ Insert code below to initialize hostInput1 and hostInput2 to random numbers, and create reference result in CPU
  for (int i = 0; i < inputLength; i++) {
    // generate a pseudo-random integer between minimum_number and max_number
    hostInput1[i] = rand() % (max_number + 1 - minimum_number) + minimum_number;
    hostInput2[i] = rand() % (max_number + 1 - minimum_number) + minimum_number;
  }

  //@@ Insert code below to allocate GPU memory here
  cudaMalloc((void **)&deviceInput1, size);
  cudaMalloc((void **)&deviceInput2, size);
  cudaMalloc((void **)&deviceOutput, size);

  //@@ Insert code to below to Copy memory to the GPU here
  // Timer Start
  timerStart = cpuSecond();

  cudaMemcpy(deviceInput1, hostInput1, size, cudaMemcpyHostToDevice);
  cudaMemcpy(deviceInput2, hostInput2, size, cudaMemcpyHostToDevice);

  // Timer Stop
  timerStop = cpuSecond();
  DataType copyToDeviceTime = timerStop - timerStart;
  printf("Data copy from Host to Device elapsed %f sec\n", copyToDeviceTime);

  //@@ Initialize the 1D grid and block dimensions here
  int dimx = 32;
  dim3 block(dimx, 1);
  dim3 grid((inputLength + block.x - 1) / block.x, 1);

  //@@ Launch the GPU Kernel here
  // Timer Start
  timerStart = cpuSecond();

  vecAdd<<<grid, block>>>(deviceInput1, deviceInput2, deviceOutput, inputLength);
  cudaDeviceSynchronize();

  // Timer Stop
  timerStop = cpuSecond();
  DataType kernelTime = timerStop - timerStart;
  printf("CUDA Kernel elapsed %f sec\n", kernelTime);

  //@@ Copy the GPU memory back to the CPU here
  // Timer Start
  timerStart = cpuSecond();

  cudaMemcpy(hostOutput, deviceOutput, size, cudaMemcpyDeviceToHost);

  // Timer Stop
  timerStop = cpuSecond();
  DataType copyToHostTime = timerStop - timerStart;
  printf("Data copy from Device to Host elapsed %f sec\n", copyToHostTime);

  //@@ Insert code below to compare the output with the reference
  double tolerance = 1.0e-14;
  DataType expected;
  for (int i = 0; i < inputLength; i++) {
    expected = hostInput1[i] + hostInput2[i];
    // if the absolute value is greater than the tolerance we have an error
    if (fabs(hostOutput[i] - expected) > tolerance) {
      printf("\nError: value of hostOutput[%d] = %f instead of %f\n\n", i, hostOutput[i], expected);
      exit(1);
    } else {
      // printf("\nOk: value of hostOutput[%d] = %f - expected: %f\n\n", i, hostOutput[i], expected);
    }
  }

  //@@ Free the GPU memory here
  cudaFree(deviceInput1);
  cudaFree(deviceInput2);
  cudaFree(deviceOutput);

  //@@ Free the CPU memory here
  free(hostInput1);
  free(hostInput2);
  free(hostOutput);

  printf("\n---------------------------------------------\n");
  printf("SUCCESS\n");

  return 0;
}



Overwriting vectorAdd_2.cu


Compile


In [ ]:
!nvcc vectorAdd.cu -o vectorAdd
!ls

sample_data  vectorAdd	vectorAdd.cu


In [ ]:
!nvcc vectorAdd_2.cu -o vectorAdd2
!ls

a.out  matMul  matMul.cu  matrix_multiplication.cu  sample_data  vectorAdd2  vectorAdd_2.cu


Running different input lengths 10 times.


In [ ]:
! ./vectorAdd2 140000
! ./vectorAdd2 150000
! ./vectorAdd2 160000
! ./vectorAdd2 170000
! ./vectorAdd2 180000
! ./vectorAdd2 190000
! ./vectorAdd2 200000
! ./vectorAdd2 210000
! ./vectorAdd2 220000
! ./vectorAdd2 230000

The input length is 140000
Data copy from Host to Device elapsed 0.000593 sec
CUDA Kernel elapsed 0.000069 sec
Data copy from Device to Host elapsed 0.000879 sec

---------------------------------------------
SUCCESS
The input length is 150000
Data copy from Host to Device elapsed 0.000617 sec
CUDA Kernel elapsed 0.000071 sec
Data copy from Device to Host elapsed 0.001026 sec

---------------------------------------------
SUCCESS
The input length is 160000
Data copy from Host to Device elapsed 0.000630 sec
CUDA Kernel elapsed 0.000067 sec
Data copy from Device to Host elapsed 0.001031 sec

---------------------------------------------
SUCCESS
The input length is 170000
Data copy from Host to Device elapsed 0.000699 sec
CUDA Kernel elapsed 0.000059 sec
Data copy from Device to Host elapsed 0.001070 sec

---------------------------------------------
SUCCESS
The input length is 180000
Data copy from Host to Device elapsed 0.000716 sec
CUDA Kernel elapsed 0.000056 sec
Data copy from Device

Code for exercise 2 - matrix multiplication.

It is currently chosen double as DataType.

In [ ]:
%%writefile matMul.cu

#include <stdio.h>
#include <sys/time.h>
#include <stdlib.h>


#define DataType float

// Compute C = A * B
__global__ void gemm(DataType *A, DataType *B, DataType *C, int numARows,
                      int numAColumns, int numBRows, int numBColumns){

	 //@@ Insert code to implement matrix multiplication here
    int rows = blockIdx.y * blockDim.y + threadIdx.y;
    int cols = blockIdx.x * blockDim.x + threadIdx.x;


    if(rows < numARows && cols < numBColumns){
        DataType sum = 0;
        for(int i = 0; i < numAColumns; i++){
            sum += A[rows * numAColumns + i] * B[i * numBColumns + cols];
        }
        C[rows * numBColumns + cols] = sum;
    }
}


int main(int argc, char **argv) {

    DataType *hostA; // The A matrix
    DataType *hostB; // The B matrix
    DataType *hostC; // The output C matrix
    DataType *resultRef; // The reference result
    DataType *deviceA;
    DataType *deviceB;
    DataType *deviceC;
    int numARows;    // number of rows in the matrix A
    int numAColumns; // number of columns in the matrix A
    int numBRows;    // number of rows in the matrix B
    int numBColumns; // number of columns in the matrix B
    int numCRows;
    int numCColumns;

    //@@ Insert code below to read in numARows, numAColumns, numBColumns and numBRows from args
    if(argc != 4){
        printf("Usage: %s numARows numAColumns numBColumns\n", argv[0]);
        exit(1);
    }
    numARows = atoi(argv[1]);
    numAColumns = atoi(argv[2]);
    numBColumns = atoi(argv[3]);
    numBRows = numAColumns;
    numCRows = numARows;
    numCColumns = numBColumns;
    if(numARows <= 0 || numAColumns <= 0 || numBColumns <= 0){
        printf("Invalid arguments! \n");
        exit(1);
    }

     printf("Input matrix dimensions: A(%d x %d), B(%d x %d), C(%d x %d)\n", numARows, numAColumns, numBRows, numBColumns, numCRows, numCColumns);


    //@@ Insert code below to allocate Host memory for input and output
    hostA = (DataType*)malloc(numARows * numAColumns * sizeof(DataType));
    hostB = (DataType*)malloc(numBRows * numBColumns * sizeof(DataType));

    //Checking in order to interrupt memory allocation didnt work.
    if(hostA == NULL || hostB == NULL){
        printf("Allocating memory failed\n");
        exit(1);
    }


    //@@ Insert code below to initialize hostA and hostB to random numbers, and create reference result in CPU



    for(int i = 0; i < numARows; i++) {
        for(int j = 0; j < numAColumns; j++) {
            hostA[i * numAColumns + j] = (DataType)rand() / RAND_MAX;
        }
    }

    for(int i = 0; i < numBRows; i++) {
        for(int j = 0; j < numBColumns; j++) {
            hostB[i * numBColumns + j] = (DataType)rand() / RAND_MAX;
        }
    }

    // Allocate memory for resultRef
    resultRef = (DataType*)malloc(numCRows * numCColumns * sizeof(DataType));
    if (resultRef == NULL) {
        printf("Error allocating memory for resultRef\n");
        exit(1);
    }

    // Compute the reference result in CPU


    for(int i = 0; i < numARows; i++) {
        for(int j = 0; j < numBColumns; j++) {
            resultRef[i * numBColumns + j] = 0; // Initialize the element to 0
            for(int k = 0; k < numAColumns; k++) {
                // Accumulate the sum for the dot product of row i from A and column j from B
                resultRef[i * numBColumns + j] += hostA[i * numAColumns + k] * hostB[k * numBColumns + j];
            }
        }
    }



    hostC = (DataType *)malloc(numCRows * numCColumns * sizeof(DataType));
    if(hostA == NULL || hostB == NULL || hostC == NULL || resultRef == NULL){
        printf("Error allocating memory\n");
        exit(1);
    }

    //@@ Insert code below to allocate GPU memory here
    cudaMalloc((void **)&deviceA, numARows * numAColumns * sizeof(DataType));
    cudaMalloc((void **)&deviceB, numBRows * numBColumns * sizeof(DataType));
    cudaMalloc((void **)&deviceC, numCRows * numCColumns * sizeof(DataType));

    //@@ Insert code to below to Copy memory to the GPU here

    timeval timer;

    gettimeofday(&timer, NULL);
    double start_memtogpu = timer.tv_sec * 1000000 + timer.tv_usec;


    cudaMemcpy(deviceA, hostA, numARows * numAColumns * sizeof(DataType), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB, numBRows * numBColumns * sizeof(DataType), cudaMemcpyHostToDevice);

    gettimeofday(&timer, NULL);
    double end_memtogpu = timer.tv_sec * 1000000 + timer.tv_usec;

    double elapsed_time_memtogpu = (end_memtogpu - start_memtogpu) / 1e6;
    printf("Copy memory to GPU: %f s\n", elapsed_time_memtogpu);

    //@@ Initialize the grid and block dimensions here
    dim3 dimGrid((numBColumns - 1) / 32 + 1, (numARows - 1) / 32 + 1, 1);
    dim3 dimBlock(32, 32, 1);

    //@@ Launch the GPU Kernel here

    gettimeofday(&timer, NULL);
    double start_gpukernel = timer.tv_sec * 1000000 + timer.tv_usec;
    gemm<<<dimGrid, dimBlock>>>(deviceA, deviceB, deviceC, numARows, numAColumns, numBRows, numBColumns);
    cudaDeviceSynchronize();

    gettimeofday(&timer, NULL);
    double end_gpukernel = timer.tv_sec * 1000000 + timer.tv_usec;

    double elapsed_time_gpukernel = (end_gpukernel - start_gpukernel) / 1e6;
    printf("Time to execute the GPU Kernel: %f s\n", elapsed_time_gpukernel);


    //@@ Copy the GPU memory back to the CPU here

    gettimeofday(&timer, NULL);
    double start_gputocpu = timer.tv_sec * 1000000 + timer.tv_usec;
    cudaMemcpy(hostC, deviceC, numCRows * numCColumns * sizeof(DataType), cudaMemcpyDeviceToHost);

    gettimeofday(&timer, NULL);
   double end_gputocpu = timer.tv_sec * 1000000 + timer.tv_usec;
    double elapsed_time_gputocpu = (end_gputocpu - start_gputocpu) / 1e6;
    printf("Time to copy from GPU memory to CPU: %f s\n", elapsed_time_gputocpu);




    //@@ Insert code below to compare the output with the reference
    bool error = false;
    for(int i = 0; i < numCRows; i++) {
        for(int j = 0; j < numCColumns; j++) {
            if(abs(hostC[i * numCColumns + j] - resultRef[i * numCColumns + j]) > 1e-2){
                error = true;
                break;
            }
        }
    }

    if(error) {
        printf("The results are incorrect!\n");
    } else {
        printf("The results are correct.\n");
    }



    //@@ Free the GPU memory here
    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);



    //@@ Free the CPU memory here
    free(hostA);
    free(hostB);
    free(hostC);
    free(resultRef);



    return 0;
}



Overwriting matMul.cu


Compile+

In [ ]:
!nvcc matMul.cu -o matMul

Run matMul 10 times, with a beginning of matrix A of 600x1200, matrix B of 1200x2400. up until matrix A 1500x3000, matrix B 3000x6000. with matrix C having 1500x6000

In [ ]:
! ./matMul 600 1200 4100
! ./matMul 700 1400 4200
! ./matMul 800 1600 4300
! ./matMul 900 1800 4400
! ./matMul 1000 2000 4500
! ./matMul 1100 2200 4600
! ./matMul 1200 2400 4700
! ./matMul 1300 2600 4800
! ./matMul 1400 2800 4900
! ./matMul 1500 3000 5000

Input matrix dimensions: A(600 x 1200), B(1200 x 4100), C(600 x 4100)
Copy memory to GPU: 0.005111 s
Time to execute the GPU Kernel: 0.018735 s
Time to copy from GPU memory to CPU: 0.006802 s
The results are correct.
Input matrix dimensions: A(700 x 1400), B(1400 x 4200), C(700 x 4200)
Copy memory to GPU: 0.005908 s
Time to execute the GPU Kernel: 0.026306 s
Time to copy from GPU memory to CPU: 0.008085 s
The results are correct.
Input matrix dimensions: A(800 x 1600), B(1600 x 4300), C(800 x 4300)
Copy memory to GPU: 0.007240 s
Time to execute the GPU Kernel: 0.034814 s
Time to copy from GPU memory to CPU: 0.009416 s
The results are correct.
Input matrix dimensions: A(900 x 1800), B(1800 x 4400), C(900 x 4400)
Copy memory to GPU: 0.008441 s
Time to execute the GPU Kernel: 0.044493 s
Time to copy from GPU memory to CPU: 0.011051 s
The results are correct.
Input matrix dimensions: A(1000 x 2000), B(2000 x 4500), C(1000 x 4500)
Copy memory to GPU: 0.009744 s
Time to execute the GPU Kerne

In [ ]:
!ls

a.out  matMul  matMul.cu  matrix_multiplication.cu  sample_data


In [ ]:
!cat /etc/os-release | grep "VERSION_ID"
!echo "Machine's architecture: `uname -i`"

VERSION_ID="22.04"
Machine's architecture: x86_64


execution with nsight for input length of 1024.

In [ ]:
! /usr/local/cuda-11/bin/nv-nsight-cu-cli ./vectorAdd 1024

executing with using nsight for input length of 131070.

In [ ]:
! /usr/local/cuda-11/bin/nv-nsight-cu-cli ./vectorAdd 131070

==ERROR== './vectorAdd' does not exist or is not an executable. Please make sure to specify the absolute path to './vectorAdd' if the executable is not in the local directory.


execution with nsight for a matrix A of (128x128) and B of (128x128)

In [ ]:
! /usr/local/cuda-11/bin/nv-nsight-cu-cli ./matMul 128 128 128

execution with nsight for a matrix A of (511x1023) and B of (1023x4094)

In [ ]:
! /usr/local/cuda-11/bin/nv-nsight-cu-cli ./matMul 511 1023 4094